# Setup

In [1]:
import requests
from langchain.retrievers import ParentDocumentRetriever
from langchain.storage import InMemoryStore
from langchain_chroma import Chroma
from langchain_community.document_loaders import TextLoader
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_core.documents import Document
import nest_asyncio
from dotenv import load_dotenv
from langchain_core.messages import (
    HumanMessage,
    SystemMessage,
)
from langchain_huggingface import ChatHuggingFace, HuggingFaceEndpoint
from langchain_ollama.llms import OllamaLLM
from langchain_openai import ChatOpenAI
from bs4 import BeautifulSoup
import re
import copy
from pydantic import BaseModel


nest_asyncio.apply()

# Load environment variables from .env file
load_dotenv()

True

# DIP Data

In [2]:
base_url = "https://search.dip.bundestag.de/api/v1"
api_key = "I9FKdCn.hbfefNWCY336dL6x62vfwNKpoN2RZ1gp21"

In [3]:
# Get all documents metadata
metadata_endpoint = "plenarprotokoll"
headers = { "Authorization": f"ApiKey {api_key}"}

def get_all_document_metadatas():   
    metadata_url = f"{base_url}/{metadata_endpoint}"
    return requests.get(metadata_url, headers=headers).json()["documents"]

In [4]:
# Get Text of one document
fulltext_endpoint = "plenarprotokoll-text"
params = {"format": "xml"}

def get_text(document_id: str) -> str:
    fulltext_url = f"{base_url}/{fulltext_endpoint}/{document_id}"
    response = requests.get(fulltext_url, headers=headers, params=params)
    return response.content


In [5]:
class Redner(BaseModel):
    last_name: str
    first_name: str
    party: str | None
    rolle: str | None
    
class Beitrag(BaseModel):
    datum: str
    rede: str
    redner: Redner

In [16]:
def get_beitrag_from_rede_tag(datum: str, rede_tag) -> Beitrag:
    # Get redner
    redner_tag = rede_tag.find('redner')
    first_name = redner_tag.find('vorname').text
    last_name = redner_tag.find('nachname').text
    party = redner_tag.find('fraktion').text if redner_tag.find('fraktion') else ""
    rolle = redner_tag.find('rolle_lang').text if redner_tag.find('rolle_lang') else ""
    redner = Redner(last_name=last_name, first_name=first_name, party=party,rolle=rolle)
    redner_tag.decompose()

    # Delete comments
    for kommentar in rede_tag.find_all('kommentar'):
        kommentar.decompose()

    return Beitrag(redner=redner, rede=rede_tag.text, datum=datum)


In [17]:
metadatas = get_all_document_metadatas()

In [26]:
index = 3
xml_url = metadatas[index]["fundstelle"]["xml_url"]
xml_content = requests.get(xml_url).content
soup = BeautifulSoup(xml_content, features='xml')

all_reden_tags = soup.find_all("rede")
beitraege = []

datum = soup.find("datum").get("date")
for rede_tag in all_reden_tags:
    rede_tag_cp = copy.deepcopy(rede_tag)
    beitrag = get_beitrag_from_rede_tag(datum=datum, rede_tag=rede_tag_cp)
    beitraege.append(beitrag)
print(len(beitraege))
print(beitraege[0].datum)

64
11.10.2024


# RAG

In [19]:
# Create ParentDocumentRetriever
embedding_model = "sentence-transformers/all-MiniLM-L6-v2"
parent_splitter = RecursiveCharacterTextSplitter(chunk_size=2000)
child_splitter = RecursiveCharacterTextSplitter(chunk_size=400)
vectorstore = Chroma(
    collection_name="split_parents", embedding_function=HuggingFaceEmbeddings(model_name=embedding_model)
)
store = InMemoryStore()

retriever = ParentDocumentRetriever(
    vectorstore=vectorstore,
    docstore=store,
    child_splitter=child_splitter,
    parent_splitter=parent_splitter,
)

In [20]:
def get_langchain_document_from_beitrag(beitrag: Beitrag) -> Document:
    redner_dict = beitrag.redner.dict()
    metadata = {}
    metadata.update(redner_dict)
    metadata.update({"datum": beitrag.datum})
    return Document(page_content=beitrag.rede, metadata=metadata)

In [21]:
documents = [get_langchain_document_from_beitrag(beitrag) for beitrag in beitraege]
retriever.add_documents(documents=documents)

In [22]:
# Load Generation Model
llama_model_3 = "meta-llama/Llama-3.2-3B-Instruct"
mixtral_model = "mistralai/Mixtral-8x7B-Instruct-v0.1"
llama_2b_70b = "meta-llama/Llama-2-70b-chat-hf"
openai_gpt_3_5 = "gpt-3.5-turbo"

llm = HuggingFaceEndpoint(
    repo_id=mixtral_model,
    task="text-generation",
    max_new_tokens=1024,
)

chat_model = ChatOpenAI(model=openai_gpt_3_5)
#chat_model = ChatHuggingFace(llm=llm)
fact_check_model = ChatHuggingFace(llm=llm)
#chat_model = OllamaLLM(model="mistral")
#fact_check_model = OllamaLLM(model="llama3.2:1b")


The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: read).
Your token has been saved to /home/tim/.cache/huggingface/token
Login successful


In [23]:
def format_docs(documents: list[Document]) -> str:
    return "\n\n".join(document.page_content for document in documents)

In [30]:
def get_rag_answer(prompt: str):
    documents = retriever.invoke(prompt,)
    context = format_docs(documents=documents)

    simple_rag_prompt = (
        "Beantworte die Frage mit Hilfe der folgenden Kontextinformationen. "
        "Wenn du die Antwort nicht weißt, sag einfach, dass du die Antwort nicht kennst. "
        "Verwende zu Beantwortung nur die Informationen im Kontext. Verwende kein externes Wissen. "
        "Verwende maximal fünf Sätze und fasse die Antwort kurz zusammen."
        f"Frage: {prompt}"
        f"Kontext: {context}"
        "Antwort: " 
        )

    simple_system_prompt =  "Du bist ein Assistent für die Beantwortung von Fragen bezüglich Plenarsitzungen des Deutschen Bundestags."

    messages = [
        SystemMessage(content=simple_system_prompt),
        HumanMessage(
            content=simple_rag_prompt
        ),
    ]

    answer = chat_model.invoke(messages).content
    return (answer, documents)

In [25]:
# Self Fact-Checking
def check_facts(answer:str, documents: list[Document])-> tuple[bool, str]:
    context = format_docs(documents=documents)

    fact_checking_prompt = (
        "Du hast die Aufgabe, herauszufinden, ob die Hypothese begründet ist und mit den Beweisen übereinstimmt. "
        "Verwende nur den Inhalt der Beweise und stütze dich nicht auf externes Wissen. "
        f"Antworte mit ja/nein. Beweise: {context} "
        f"Hypothese: {answer}: "
        "Antwort: "
    )

    ai_msg = fact_check_model.invoke([
        HumanMessage(
            content=fact_checking_prompt
        ),
    ]).content
    is_okay = ai_msg.lower().strip().startswith("ja")
    return (is_okay, ai_msg)


In [31]:
# Ask RAG
prompt = "Was sagt die SPD zur Stärkung der Strukturen gegen sexuelle Gewalt gegen Kinder und Jugendliche??"
(answer, documents) = get_rag_answer(prompt=prompt)
(is_based_on_facts, fact_checking_answer) = check_facts(answer=answer, documents=documents)

print(f"Fact-Check: {'OKAY' if is_based_on_facts else 'NOT OKAY'}")
print(answer)

{}
Fact-Check: OKAY
Die SPD unterstützt das Gesetz zur Stärkung der Strukturen gegen sexuelle Gewalt an Kindern und Jugendlichen, betont jedoch die Notwendigkeit von Nachbesserungen, insbesondere hinsichtlich der Verfügbarkeit und Finanzierung von Beratungsstellen sowie der Klarheit in Begrifflichkeiten und Zuständigkeiten. Die Partei begrüßt die geplanten Maßnahmen zur Stärkung der Prävention durch Schulungen von Fachkräften in der Jugendhilfe, Kitas, Schulen, Jugendämtern und gesundheitlichen Einrichtungen. Zudem betont die SPD die Wichtigkeit der Einbindung und Stärkung der Rechte von Betroffenen durch den verankerten Betroffenenrat und die Aufarbeitungskommission.


In [13]:
fact_checking_answer

' Ja, die Hypothese wird bestätigt. Aus dem Stenografischen Bericht geht hervor, dass Stephan Brandner (AfD) Herrn Steffen als "Hetzer" bezeichnet hat.'

In [32]:
len(documents)

3

In [ ]:
# Build Evaluation Dataset

evaluation_questions = [
    "Wer nannte Herrn Steffen einen Hetzer?"
]

evaluation_answers = [
    "Stephan Brandner (AfD) nannte Herrn Steffen einen 'Hetzer'."
]